# Dremio MinIO Iceberg


In this notebook we will setup Dremio to read files like CSV from Minio, we will also access Apache Iceberg table that was created using Spark as shown [here](../spark/spark-iceberg-minio.ipynb). If you haven't setup Dremio yet follow the walkthrough in [this Notebook](setup-dremio.ipynb)

## Add MinIO as Datasource

Once we are login to Dremio, lets click on `Add Source` at the bottom left

![add_source](./img/add-source.png)

Select `Amazon S3` under `Object Storage`

![select_s3](./img/select-S3.png)

Fill in the details like `Name` of the connector `AWS Access Key`, `AWS Access Secret` and add `Buckets` in our case `openlake` as shown below

![s3-details](./img/s3-details.png)

Next we choose the `Advanced Options` on the left side of the menu enable `Enable compatibility mode` and add 2 new `Connection Properties`
* fs.s3a.endpoint - play.min.io
* fs.s3a.path.style.access - true

Add `openlake` to the `Allowlisted buckets` and hit `save` as shown in the image below

![s3-details2](./img/s3-details2.png)

## Accessing CSV file

Let's use the `taxi-data.csv` that we used in spark notebooks, if not you can follow the instructions [here](../spark/spark-iceberg-minio.ipynb#Getting-Demo-Data-into-MinIO) to get the data into MinIO. Click on the `openlake` datasource that we just setup

![source](./img/source.png)

Navigate to `openlake/spark/sample-data` you should see `taxi-data.csv` file there click on the `Format File` as shown below

![format-file](./img/format-file.png)

Dremio should be able to infer the schema of the CSV file but do some changes as shown below

![schema](./img/schema.png)

Click on `Save` and you should be navigated to the SQL editor, lets run a simple query to see the data

```sql
SELECT count(*) FROM openlake.openlake.spark."sample-data"."taxi-data.csv";
```

It will take sometime to load the data and compute the count, once done you should see the result as shown below

![count](./img/count.png)
Note: It took approx more than 2 mins to complete the above query, the time taken depends on the size of the data and the compute resources available to Dremio.

We can perform other query operations but we will not be able to alter the column names or time travel to previous versions of the data. We will use Apache Iceberg to do that.

## Accessing Iceberg table

We will use the `nyc.taxis_large` Iceberg table that we created in [this Notebook](../spark/spark-iceberg-minio.ipynb) to access the data using Dremio. Click on the `openlake` datasource that we just setup

![source](./img/source.png)

Navigate to `openlake/warehouse/nyc` you should see `taxis_large` there click on the `Format File` Dremio should be able to infer the schema of the Iceberg table as shown below

![iceberg_schema](./img/iceberg-schema.png)

Click on `Save` and you should be navigated to the SQL editor, lets run a simple query to see the data

```sql
SELECT count(*) FROM openlake.openlake.warehouse.nyc.taxis_large;
```
The query execution time is much faster than the CSV file `<1s` as shown below
![count_iceberg](./img/count-iceberg.png)

Note: we are seeing all the records in the table since we switched to the earliest snapshot of the table via time travel from the `spark-iceberg-minio` [notebook](../spark/spark-iceberg-minio.ipynb#T#Time-Travel-with-snapshots)

At the time of this writing Dremio does not support time travel via snapshots or to access any metadata tables, so we will not be able to access the data in the previous snapshots. We can use processing engines like Spark to access the data in previous snapshots.

### Update Table Operations
Let's try to populate the `fare_per_distance` column by dividing the `fare` by `trip` and see if we can query the data, You can execute the following queries in the SQL editor all at once

```sql
SELECT VendorID
        ,tpep_pickup_datetime
        ,tpep_dropoff_datetime
        ,fare
        ,distance
        ,fare_per_distance
        FROM openlake.openlake.warehouse.nyc.taxis_large LIMIT 15;

UPDATE openlake.openlake.warehouse.nyc.taxis_large SET fare_per_distance = fare/distance WHERE distance > 0;

SELECT VendorID
        ,tpep_pickup_datetime
        ,tpep_dropoff_datetime
        ,fare
        ,distance
        ,fare_per_distance
        FROM openlake.openlake.warehouse.nyc.taxis_large LIMIT 15;
```
Above we are first fetching first 15 records from the Iceberg table there we will notice `fare_per_distance` is `null`, then we execute the `UPDATE` query to populate the `fare_per_distance` column and finally we fetch the first 15 records again and we will notice the `fare_per_distance` column is populated.

![query-1](./img/query-1-iceberg.png)

![query-2](./img/query-2-iceberg.png)

![query-3](./img/query-3-iceberg.png)


### Delete Table Operations
Let's try to delete some records from the table, You can execute the following queries in the SQL editor all at once

```sql
DELETE FROM openlake.openlake.warehouse.nyc.taxis_large WHERE fare_per_distance > 4.0 OR distance > 2.0;
DELETE FROM openlake.openlake.warehouse.nyc.taxis_large WHERE fare_per_distance IS NULL;

SELECT count(*) FROM openlake.openlake.warehouse.nyc.taxis_large;
```

![query-4](./img/query-4-iceberg.png)

![query-5](./img/query-5-iceberg.png)

![query-6](./img/query-6-iceberg.png)

Above we are deleting records where `fare_per_distance` is greater than 4.0 or `distance` is greater than 2.0 and then we are deleting records where `fare_per_distance` is `null` and finally we are counting the number of records in the table.

### Monitor Jobs

We can monitor the jobs that are running or already completed in Dremio by clicking on the `Jobs` tab on the left side of the menu as shown below

![jobs](./img/jobs.png)
Jobs can be filtered by based on various parameters depending on the user need.

We can also see the details of the job like the memory usage breakdown of the total execution time etc., by clicking on the job name as shown below.
![job-details](./img/job-details.png)


We have now setup Dremio to read data from MinIO and access Iceberg table created using Spark.